### 背景

罗斯福死于高血压。美国政府因此加强对心血管疾病的研究。因此，Framingham心脏研究跟踪了一组病人，作为研究对象。

>  Framingham had an appropriate size, it had a stable population, and the doctors and residents in the town were willing to participate. However, the city did not represent all types of people in the United States (we'll see later in the lecture how to extend the model to different populations) and there were not an abnormally large number of people with heart disease.

FHD(Framingham Heart Study)为日后的心血管疾病的治疗做出很大贡献。例如，冠状动脉疾病（CHD）的死亡率大大下降，得益于FHD的研究成果，它使得冠状动脉疾病能得到更早的发现。

重要的**风险因子**的发现是极为重要的。例如，**吸烟**这一风险因子，在40年代是很新奇的说法。

### 用风险因子预测10年的CHD风险

In [1]:
framingham = read.csv("./data/framingham.csv")
str(framingham)

'data.frame':	4240 obs. of  16 variables:
 $ male           : int  1 0 1 0 0 0 0 0 1 1 ...
 $ age            : int  39 46 48 61 46 43 63 45 52 43 ...
 $ education      : int  4 2 1 3 3 2 1 2 1 1 ...
 $ currentSmoker  : int  0 0 1 1 1 0 0 1 0 1 ...
 $ cigsPerDay     : int  0 0 20 30 23 0 0 20 0 30 ...
 $ BPMeds         : int  0 0 0 0 0 0 0 0 0 0 ...
 $ prevalentStroke: int  0 0 0 0 0 0 0 0 0 0 ...
 $ prevalentHyp   : int  0 0 0 1 0 1 0 0 1 1 ...
 $ diabetes       : int  0 0 0 0 0 0 0 0 0 0 ...
 $ totChol        : int  195 250 245 225 285 228 205 313 260 225 ...
 $ sysBP          : num  106 121 128 150 130 ...
 $ diaBP          : num  70 81 80 95 84 110 71 71 89 107 ...
 $ BMI            : num  27 28.7 25.3 28.6 23.1 ...
 $ heartRate      : int  80 95 75 65 85 77 60 79 76 93 ...
 $ glucose        : int  77 76 70 103 85 99 85 78 79 88 ...
 $ TenYearCHD     : int  0 0 0 1 0 0 1 0 0 0 ...


In [2]:
library("caTools")
set.seed(1000)
split = sample.split(framingham$TenYearCHD, SplitRatio=0.65)

In [3]:
train = subset(framingham, split==TRUE)
test = subset(framingham, split=FALSE)

In [4]:
framinghamLog = glm(TenYearCHD ~ ., data=train, family=binomial)
summary(framinghamLog)


Call:
glm(formula = TenYearCHD ~ ., family = binomial, data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8487  -0.6007  -0.4257  -0.2842   2.8369  

Coefficients:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -7.886574   0.890729  -8.854  < 2e-16 ***
male             0.528457   0.135443   3.902 9.55e-05 ***
age              0.062055   0.008343   7.438 1.02e-13 ***
education       -0.058923   0.062430  -0.944  0.34525    
currentSmoker    0.093240   0.194008   0.481  0.63080    
cigsPerDay       0.015008   0.007826   1.918  0.05514 .  
BPMeds           0.311221   0.287408   1.083  0.27887    
prevalentStroke  1.165794   0.571215   2.041  0.04126 *  
prevalentHyp     0.315818   0.171765   1.839  0.06596 .  
diabetes        -0.421494   0.407990  -1.033  0.30156    
totChol          0.003835   0.001377   2.786  0.00533 ** 
sysBP            0.011344   0.004566   2.485  0.01297 *  
diaBP           -0.004740   0.008001  -0.592  0

所有显著的因子系数都为正，合理。用这个模型做初步预测。

In [6]:
predictTest = predict(framinghamLog, type="response", newdata=test)
table(test$TenYearCHD, predictTest > 0.5)

   
    FALSE TRUE
  0  3088   13
  1   521   36

可以看出，把阈值设为0.5，则很少预测“CHD风险高于0.5”。模型accuracy：

In [7]:
(3088+36)/(3088+13+521+36)

[1] 0.8540186

基准模型准确率：

In [8]:
(3088+13)/(3088+13+521+36)

[1] 0.847731

这个模型的准确率和基准模型差不多。计算该模型的AUC。

In [11]:
library("ROCR")
ROCRpred = prediction(predictTest, test$TenYearCHD)
as.numeric(performance(ROCRpred, "auc")@y.values)

[1] 0.737577

得到AUC值为0.74，说明模型在样本外区分高风险病人和低风险病人的能力强。